In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from utils import printMatrix
from matplotlib.animation import FuncAnimation

plt.style.use('seaborn-pastel')

In [ ]:
k = 57e6 # N/m
m = 20e3 # kg

k1, k2, k3, k4 = k, k, k, k
m1, m2, m3, m4 = m, m, m, m

K = np.array([[k1+k2, -k2, 0, 0],
             [-k2, k2+k3, -k3, 0],
             [0, -k3, k3+k4, -k4],
             [0, 0, -k4, k4]])

M = np.diag((m1, m2, m3, m4))
print(('K = {}').format(K))
print(('M = {}').format(M))

In [ ]:
A = np.linalg.inv(M)@K
lamb, eigv = np.linalg.eig(A)

In [ ]:
w = np.sqrt(lamb)
f = w/2/np.pi
print(('f = {}').format(f))

In [ ]:
print(('v = {}').format(eigv))

In [ ]:
M_norm = np.transpose(v)@M@eigv
M_norm[M_norm<1e-3] = np.nan
printMatrix(M_norm)

In [ ]:
K_norm = np.transpose(v)@K@eigv
K_norm[K_norm<1e-3] = np.nan
printMatrix(K_norm)

In [ ]:
w_2 = np.zeros(len(w))
for mode in range(len(w)):
    w_2[mode] = np.sqrt(K_norm[mode,mode]/M_norm[mode,mode])
f_2 = w_2/np.pi/2
print(('f2 = {}').format(f_2))

In [ ]:
A = np.linalg.inv(M)@K
print(('Av = {}').format(A@eigv[:,0]))
print(('wv = {}').format(lamb[0]*eigv[:,0]))

In [ ]:
for mode in range(len(w)):
    eigv[:,mode] = eigv[:,mode]/max(abs(eigv[:,mode]))
eigv

In [ ]:
printMatrix(eigv)

In [ ]:
fig, ax = plt.subplots()
fig.add_axes()
ax.set(xlim=[-1,1], ylim=[0,5], )
ax.grid()
ax.set_aspect('equal')
eigv_plot = np.zeros((eigv.shape[1] + 1, eigv.shape[1]))
eigv_plot[1:, :] = eigv

ax.plot(np.zeros(eigv.shape[1] + 1),np.linspace(0,4,5), color='k', marker='o')
for col in range(eigv.shape[1]):
    ax.plot(eigv_plot[:,col],[0, 1, 2, 3, 4], marker='o')
    
plt.show()

In [ ]:
fig, ax = plt.subplots()
fig.add_axes()
ax.set(xlim=[-1,1], ylim=[0,5], )
ax.grid()
ax.set_aspect('equal')

line1, = ax.plot([], [], lw=3)
line2, = ax.plot([], [], lw=3)
line3, = ax.plot([], [], lw=3)
line4, = ax.plot([], [], lw=3)

def init():
    line1.set_data([], [])
    line2.set_data([], [])
    line3.set_data([], [])
    line4.set_data([], [])
    return line1, line2, line3, line4

def animate(i):
    y = np.linspace(0, 4, 5)
    line1.set_data(eigv_plot[:,0]*np.sin(0.2*np.pi*i), y)
    line2.set_data(eigv_plot[:,1]*np.sin(0.2*np.pi*i), y)
    line3.set_data(eigv_plot[:,2]*np.sin(0.2*np.pi*i), y)
    line4.set_data(eigv_plot[:,3]*np.sin(0.2*np.pi*i), y)
    return line1, line2, line3, line4

anim = FuncAnimation(fig, animate, init_func=init,
                               frames=10, interval=200, blit=True)

anim.save('sine_wave.gif', writer='imagemagick')